# 檔頭

In [1]:
import os
import re
import cv2
import time
import torch
import random
import one_D_RBF
import two_D_RBF_v2 as two_D_RBF
import clean_values
import plot
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import trange
from random import randrange
from datetime import date
from datetime import datetime
from datetime import timedelta

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Flatten,Conv2D,BatchNormalization,Activation,GlobalAveragePooling2D,concatenate, AveragePooling2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Input, Concatenate,TimeDistributed,Lambda,SimpleRNN,LSTM,Dropout,Permute,Reshape
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.regularizers import L1L2

# strategy = tf.distribute.MirroredStrategy()

# 讀資料(雨量、潮汐、河川水位) + 設定資料區間

In [34]:
# region = '_高雄'
region = '_宜蘭'
region_en = 'Yilan'
def fin_spdata(datatype, P):
    path = P
    data_list = os.listdir(path)
    data_list = " ".join(data_list)  
    find_specific_type_data = re.findall(r'\S+.' + datatype, data_list)
    return find_specific_type_data
P = 'C:/Users/User/01.災防淹水{}/'.format(region)
dataset = fin_spdata('csv',P)
dataset

['flood.csv', 'rain.csv', 'river.csv', 'tide.csv']

In [50]:
tide = pd.read_csv(P+'/'+'tide.csv',index_col=[0])
rain = pd.read_csv(P+'/'+'rain.csv',index_col=[0]) 
river = pd.read_csv(P+'/'+'river.csv',index_col=[0])
event = pd.read_csv(P+'flood.csv',index_col=[0])

# 河川

In [55]:
river
#一般正規化(each_column)
def each_normalization(data):
    original = data.copy()
    cnames = list(data.columns)[1:]
    for cname in cnames :
        data[cname] = (data[cname] - data[cname].min()) / (data[cname].max() - data[cname].min())
    return original, data
river_V2, river_V2_norm = each_normalization(river)

In [97]:
river_V2_norm['time'] = pd.to_datetime(river_V2_norm['time'], format="%Y-%m-%d %H:%M:%S")
# river_V2_norm.head(2)

In [58]:
river_V2_norm
show = plot.mat_plot(river_V2_norm.iloc[:,1:],x_name='time',y_name='river height', title='river in {}'.format(region_en))
show.plotly_plot()

# 潮汐

In [59]:
tide
#潮汐一般正規化(each_column)
def each_normalization_v2(data):
    original = data.copy()
    cnames = list(data.columns)[1:]
    for cname in cnames :
        if cname == 'time':
            continue
        else:
            data[cname] = (data[cname] - data[cname].min()) / (data[cname].max() - data[cname].min())
    return original, data
tide_v2, tide_v2_norm = each_normalization_v2(tide)

In [98]:
tide_v2_norm['time'] = pd.to_datetime(tide_v2_norm['time'], format="%Y-%m-%d %H:%M:%S")

In [61]:
tide_v2_norm
show = plot.mat_plot(tide_v2_norm.iloc[:,1:],x_name='time',y_name='tide height', title='tide in {}'.format(region_en))
show.plotly_plot()

# 雨量

In [73]:
def global_normalization(data):
    original = data.copy()
    cnames = list(data.columns)
    Max = data.iloc[:,1:].max().max()
    for cname in cnames[1:] :
        data[cname] = data[cname] / Max
    return original, data
rain, rain_norm = global_normalization(rain)

In [99]:
rain_norm['time'] = pd.to_datetime(rain_norm['time'], format="%Y-%m-%d %H:%M:%S")

In [76]:
rain_norm
show = plot.mat_plot(rain_norm.iloc[:,1:],x_name='time',y_name='rain height', title='rain in {}'.format(region_en))
show.plotly_plot()

# 讀淹水資料

In [83]:
event = pd.read_csv(P+'flood.csv',index_col=[0])
event['Stage'] = list(event.iloc[:,1:].max(axis=1))
#2小時最大淹水高度
twoH_Stage = []
def feauture_H_max_Stage(event=event,n=2):
    for i in range(len(event)):
        if i <= len(event)-n:
            flood_max = max(event['Stage'][i:i+n])
            twoH_Stage.append(flood_max)
        else:
            twoH_Stage.append(0)
    event['{}H_Stage'.format(n)] = twoH_Stage
    return event
event = feauture_H_max_Stage(event=event,n=2)
# event

In [84]:
show = plot.mat_plot(event.iloc[:,1:],x_name='time',y_name='flood height', title='flooding in {}'.format(region_en))
show.plotly_plot()

In [85]:
event_list = event.copy()
#
filter1 = event_list['time'] <= '2020-08-30'
filter2 = event_list['Stage'] > 0
filter3 = event_list['Stage'] == 0
filter2_2H = event_list['2H_Stage'] > 0
filter3_2H = event_list['2H_Stage'] == 0
print('before 2020-09-30 & flood_2H>0　:',len(event_list[(filter1 & filter2_2H)]))
print('before 2020-09-30 & flood==0　:',len(event_list[(filter1 & filter3)]))
event_ori = event_list.copy()
event_list_flood = event_list[(filter1&filter2_2H)]
event_list_peace = event_list[(filter1&filter3_2H)]
event_list_flood = event_list_flood.reset_index(drop=True)
event_list_peace = event_list_peace.reset_index(drop=True)
print(len(event_list_flood))
print(len(event_list_peace))

before 2020-09-30 & flood_2H>0　: 1343
before 2020-09-30 & flood==0　: 343
1343
337


In [47]:
# event_list_flood
# event_list_peace

In [ ]:
# def rd_sampling(data,quantity=len(event_list_flood)):
#     ##########等距抽樣##########
#     # 指定抽樣數量
#     sample_count = quantity
#     # 獲取最大樣本量
#     record_count = data.shape[0]
#     # 抽樣間距
#     width = record_count//sample_count
#     data_sample = pd.DataFrame()
#     i = 1
#     # 本量小於等於指定抽樣數量並且矩陣索引在有效範圍內是
#     while len(data_sample) <= sample_count and i * width <= record_count -1:
#         data_sample = data_sample.append(data.iloc[i*width,:])
#         i += 1
#     len(data_sample)
#     data_sample = data_sample.reset_index(drop=True)
#     return data_sample
# event_list_v2 = rd_sampling(data=event_list_peace, quantity=len(event_list_flood))

In [106]:
look_back = 3
event_list_v3 = pd.DataFrame()
event_list_v3['time'] = pd.to_datetime(event['time'], format="%Y-%m-%d %H:%M:%S")
event_list_v3['Stage'] = event['Stage']
event_list_v3 = event_list_v3.iloc[look_back:,]
event_list_v3 = event_list_v3.reset_index(drop=True)

# 抓1D資料

In [108]:
def get_feature(feature_data, time_now, look_back=12):
    time_past = time_now - timedelta(hours=look_back)
    filter1 = feature_data['time'] < time_now
    filter2 = feature_data['time'] >= time_past
    filter_data = feature_data[(filter1 & filter2)]
    return filter_data
def concat_x_y(hour, river_V2_norm, rain_Yunlin, tide_v2_norm, event_list):
    stationQTY = len(river_V2_norm.columns)+len(rain_Yunlin.columns)+len(tide_v2_norm.columns)-3
    one_D_data = np.empty((len(event_list), hour, stationQTY, 1))
    for i in range(len(event_list)):
        event_time = event_list['time'][i]
        rain_input = get_feature(feature_data=river_V2_norm, time_now=event_time, look_back=hour)
        river_input = get_feature(feature_data=rain_Yunlin, time_now=event_time, look_back=hour)
        tide_input = get_feature(feature_data=tide_v2_norm, time_now=event_time, look_back=hour)
        merge_data = pd.merge(rain_input, river_input, on='time')
        merge_data = pd.merge(merge_data, tide_input, on='time')

        merge_data = merge_data.iloc[:,1:]
        merge_data = np.array(merge_data)
#         print(np.shape(merge_data))
        merge_data = np.reshape(merge_data, (hour, stationQTY, 1))
        one_D_data[i] = merge_data
    return one_D_data
one_D_data = concat_x_y(hour=look_back, river_V2_norm=river_V2_norm, rain_Yunlin=rain_norm, tide_v2_norm=tide_v2_norm , event_list=event_list_v3)

In [111]:
np.shape(one_D_data)
one_D_data.max()
type(one_D_data)

numpy.ndarray

# 抓2D資料

In [113]:
from PIL import Image
img_dir = 'C:/Users/User/01.災防淹水{}/{}/'.format(region,region_en)
filename = os.listdir(img_dir)
arr = []
for i in filename:
    image1 = Image.open(img_dir+'/'+i)
    arr1 = np.array(image1)
    arr.append(arr1)
data = np.stack(arr)

img = fin_spdata('png',img_dir)
print(len(img))
img_time = []
for i in img:
    img_time.append(i[:10])
img_time
img_if = pd.DataFrame()
img_if['time'] = pd.to_datetime(img_time, format="%Y%m%d%H")
img_if['img'] = list(data)

2928


In [119]:
def get_feature(feature_data, time_now, look_back=12):
    time_past = time_now - timedelta(hours=look_back)
    filter1 = feature_data['time'] < time_now
    filter2 = feature_data['time'] >= time_past
    filter_data = feature_data[(filter1 & filter2)]
    return filter_data
def pick_img(hour, img_data, Width=28, Length=47, Channel=3, event_list=event_list_v3):
    two_D_data = np.empty((len(event_list), hour, Width, Length, Channel))
    for i in range(len(event_list)):
        event_time = event_list['time'][i]
        img_input = get_feature(feature_data=img_data, time_now=event_time, look_back=hour)
        img_input = img_input.reset_index(drop=True)
        each = []
        for a in range(len(img_input['img'])):
            each.append(img_input['img'][a])
        two_D_data[i] = np.array(each)
    return two_D_data

In [121]:
shape = np.shape(img_if['img'][0]) #shape = (16, 11, 3)
hour=look_back
Width=shape[0]
Length=shape[1]
Channel=shape[2]
two_D_data = pick_img(hour, img_if, Width, Length, Channel, event_list=event_list_v3)

In [124]:
np.shape(two_D_data)
two_D_data_norm = two_D_data/255.
np.shape(two_D_data_norm)

(1677, 3, 16, 11, 3)

# Shuffle

In [135]:
print(np.shape(one_D_data))
print(type(one_D_data))
print(np.shape(two_D_data))
print(np.shape(event_list_v3))

(1677, 3, 41, 1)
<class 'numpy.ndarray'>
(1677, 3, 16, 11, 3)
(1677, 2)


In [136]:
index = np.arange(len(one_D_data))
np.random.shuffle(index)
def shuffle(data, index=index):
    data_shuffle = np.ones(np.shape(data))
    for i in range(len(index)):
        data_shuffle[i] = data[index[i]]
    return data, data_shuffle
one_D_data_v2_ori, one_D_data_v2 = shuffle(one_D_data)
two_D_data_v2_ori, two_D_data_v2 = shuffle(two_D_data)
target_v2_ori, target_v2 = shuffle(np.array(event_list_v3.iloc[:,1:]))
#存洗牌後資訊
data = event_list_v3.T
data_shuffle = pd.DataFrame()
for i in range(len(index)):
    data_shuffle[i] = data[index[i]]

In [138]:
print(np.shape(one_D_data_v2))
print(np.shape(two_D_data_v2))
print(np.shape(target_v2))

(1677, 3, 41, 1)
(1677, 3, 16, 11, 3)
(1677, 1)


In [143]:
#儲存這次洗牌的樣本
data_shuffle.to_csv('data/huffle_info.csv')
np.save('data/one_D_data_v2_shuffle', one_D_data_v2)
np.save('data/two_D_data_v2_shuffle', two_D_data_v2)
np.save('data/target_v2_shuffle', target_v2)